# Oefeningen

## Breast Cancer

In [32]:
#import ...
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

In [33]:
# Laad de dataset in. Bekijk deze even om te zien waar je juist mee gaan werken
breast_cancer = load_breast_cancer()

x = breast_cancer.data
y = breast_cancer.target

In [34]:
# Split de dataset. We gebruiken in deze les consequent random seed 10.
x_trn, x_tst, y_trn, y_tst = train_test_split(x,y, test_size=0.2, random_state = 10)

In [35]:
# Gebruik een standardscaler
scaler = StandardScaler()
x_trn_scl = scaler.fit_transform(x_trn)
x_tst_scl = scaler.transform(x_tst)

In [41]:
# Creëer en train een SVM classifier. Gebruik random seed 10.
param_grid = {
    'C': [0.01,0.1, 1, 10, 100], 
    'gamma': [1, 0.1, 0.01, 0.001,0.0001], 
    'kernel': ['linear','rbf', 'poly', 'sigmoid',]  
}

classifier = SVC(random_state=10)

grid_search = GridSearchCV(estimator=classifier, 
                           param_grid=param_grid, 
                           cv=10,
                           n_jobs=-1
                          )


grid_search.fit(x_trn_scl,y_trn)
best_params = grid_search.best_params_
print(f"Best Parameters: {best_params}")
best_classifier = grid_search.best_estimator_

Best Parameters: {'C': 0.1, 'gamma': 1, 'kernel': 'linear'}


In [42]:
# Maak voorspellingen over de test data
predicted = best_classifier.predict(x_tst_scl)

In [43]:
# Bereken de accuraatheidscore van je model
accuracy = accuracy_score(y_tst,predicted)
print(accuracy)
# Laat een classification report genereren.
report = classification_report(y_tst,predicted)
print(report)

0.9736842105263158
              precision    recall  f1-score   support

           0       0.97      0.95      0.96        39
           1       0.97      0.99      0.98        75

    accuracy                           0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114


Even opfrissen:
- *Precision*: (aantal True Positives) / (aantal True Positives + aantal False Positives)
- *Recall*: (aantal True Positives) / (aantal True Positives + aantal False Negatives)
- *Accuracy*: (aantal True Positive + aantal True Negative) / (totaal aantal Positive + totaal aantal Negative)

In [ ]:
# Extra oefening: je kan nog finetunen met de hyperparameters...

## Digits

Experimenteer met verschillende kernels, C-waardes, en kies een optimale configuratie.

In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
# Digits dataset:
digits = datasets.load_digits()

# laad de data in goede variabelen

In [ ]:
# Data splitten

In [ ]:
# Standardizeren, doe dit eens in een pipeline samen met je model


In [ ]:
# Maak voorspellingen op de test data

In [ ]:

# bereken accuraatheid en classificatie report. Welk cijfers is het moeilijkst om te classificeren ?

Voor de volgende opgave mag je de hulp van ChatGPT en dergelijke inroepen.

In [ ]:
# Maak een plot van de eerste 9 testcases, waarbij je het echte label in de titel vergelijk met het voorspelde label.
# de bedoeling is dat je zo zelf visueel kan checken of de classifier het voor de specifieke items juist had.
# pas je code aan zodat er enkel testcases worden getoond die fout werden geclassificeerd.


## Wijn

Train een SVM-classificator op de Wine-dataset, die je kunt laden met 'sklearn.datasets.load_wine()'. Deze dataset bevat de chemische analyse van 178 wijnmonsters geproduceerd door 3 verschillende telers: het doel is om een classificatiemodel te trainen dat in staat is om de cultivator te voorspellen op basis van de chemische analyse van de wijn. Aangezien SVM-classifiers binaire classifiers zijn, moet je one-versus-all gebruiken om alle 3 klassen te classificeren. Welke nauwkeurigheid kan je bereiken?

Stappenplan:
1. Data inladen
2. Train/test split
3. Een lineaire SVM gebruiken
4. Een lineaire SVM gebruiken met 1000000 iteraties
5. Evalueren
6. Oplossingen zoeken voor mogelijke problemen

In [ ]:
# Lineaire SVM gebruiken
# Het probleem met meerdere klasses oplossen - bekijk de decoumentatie van LinearSVM op scikit learn docs

In [ ]:
# gebruik 1000000 iteraties
# wat is nu je conclusie over de dataset ? Hoe kun je dit oplossen ?

In [ ]:
# zoek op de documentatie op hoe je de cross_val_score kan gebruiken om cross-validatie te doen
from sklearn.model_selection import cross_val_score


### Nu kijken we naar andere kernels voor algemene SVM

We werken verder met dezelfde dataset, maar nu niet meer beperkt tot een lineaire SVM

In [ ]:
from sklearn.pipeline import Pipeline, make_pipeline # ter illustratie, make_pipeline heeft lichtjes andersyntax dan met Pipeline werken
svm_clf = make_pipeline(StandardScaler(), SVC(random_state=10))
cross_val_score(svm_clf, X_train, y_train).mean()

Nu gaan ze zoeken naar optimizatie van parameters. Dit moet ik ook nog introduceren in de theorieles

In [ ]:
# optimaliseer de parameters met CVGrid Search

Vergelijk nu de best_score_component van de CVGridSearch met de de score die dit model haalt op de test set. Er zit een verschil op. Wat is er gebeurt ? Hoe hadden we dit moeten oplossen ?

## Regressie met een SVM

Gegeven een dataset, kan je regressie doen in plaats van classificatie. De algoritmes werken bijna hetzelfde, met volgende cruciaal verschil:
- de classifier wil zoveel mogelijk instances *buiten* de straat zetten.
- de regressor wil zoveel mogelijk instances *binnen* de straat zetten.

Beschouw volgende dataset die je wil benaderen met een regressiemodel:

In [ ]:
from sklearn.svm import SVR # de R in SVR is van regressor, de C in SVC van classifier
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler

# code om een kwadratische dataset te genereren
np.random.seed(42)
X = 2 * np.random.rand(50, 1) - 1
y = 0.2 + 0.1 * X[:, 0] + 0.5 * X[:, 0] ** 2 + np.random.randn(50) / 10

In [ ]:
# maak een pipeline van een standaardscaler en de voor hand liggende SVR voor dit probleem.

Met volgende code kun je bekijken wat het model heeft bereikt (ter illustratie)

In [ ]:

import matplotlib.pyplot as plt

def find_support_vectors(svm_reg, X, y):
    y_pred = svm_reg.predict(X)
    epsilon = svm_reg[-1].epsilon
    off_margin = np.abs(y - y_pred) >= epsilon
    return np.argwhere(off_margin)

def plot_svm_regression(svm_reg, X, y, axes):
    x1s = np.linspace(axes[0], axes[1], 100).reshape(100, 1)
    y_pred = svm_reg.predict(x1s)
    epsilon = svm_reg[-1].epsilon
    plt.plot(x1s, y_pred, "k-", linewidth=2, label=r"$\hat{y}$", zorder=-2)
    plt.plot(x1s, y_pred + epsilon, "k--", zorder=-2)
    plt.plot(x1s, y_pred - epsilon, "k--", zorder=-2)
    plt.scatter(X[svm_reg._support], y[svm_reg._support], s=180,
                facecolors='#AAA', zorder=-1)
    plt.plot(X, y, "bo")
    plt.xlabel("$x_1$")
    plt.legend(loc="upper left")
    plt.axis(axes)

svm_poly_reg._support = find_support_vectors(svm_poly_reg, X, y)

fig, axes = plt.subplots(ncols=1, figsize=(9, 4), sharey=True)
plt.sca(axes)
plot_svm_regression(svm_poly_reg, X, y, [-1, 1, 0, 1])
plt.title(f"degree={svm_poly_reg[-1].degree}, "
          f"C={svm_poly_reg[-1].C}, "
          f"epsilon={svm_poly_reg[-1].epsilon}")
plt.ylabel("$y$", rotation=0)
plt.grid()

save_fig("svm_with_polynomial_kernel_plot")
plt.show()

Experimenteer met andere kernels en hperparameters of je betere resultaten kan bekomen.